# Escalamiento de los datos

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
%matplotlib inline

In [58]:
X = pd.read_csv('..\..\..\DataSets\Resultados intermedios peliculas DS\X.csv')
y = X['worldwide_gross']
X = X.drop('worldwide_gross', axis=1)
X = X.drop('gross', axis=1)

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [6]:
from sklearn.preprocessing import StandardScaler

In [7]:
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [8]:
scaler.mean_#media

array([3.34437412e+07, 2.00227319e+03, 2.12498774e+00, 1.08420603e+02,
       4.14792842e+07, 6.47121507e+00])

In [9]:
scaler.scale_#desviacion estandar

array([4.12702994e+07, 1.19917756e+01, 6.75421492e-01, 2.25146126e+01,
       2.31896422e+08, 1.06182956e+00])

In [10]:
scaler.transform(X_train)

array([[ 7.64623936e-01,  3.10780570e-01, -4.07135013e-01,
         9.14046221e-01,  9.28031389e-02, -4.43776564e-01],
       [ 4.98088433e-01,  6.06091120e-02,  3.33143473e-01,
         5.58721430e-01,  5.39927082e-02, -3.83415119e+00],
       [ 2.07322430e-01,  7.27733001e-01,  3.33143473e-01,
        -3.74006146e-01,  2.24546735e-03, -1.00883900e+00],
       ...,
       [ 1.58861430e-01, -3.56343319e-01,  3.33143473e-01,
        -1.86813551e-02, -6.37907280e-03, -3.49599491e-01],
       [ 7.64623936e-01, -1.89562346e-01, -4.07135013e-01,
        -1.96343750e-01,  7.98663287e-02, -3.49599491e-01],
       [-6.52860328e-01, -7.73295750e-01, -4.07135013e-01,
        -1.86813551e-02, -1.50840120e-01, -2.55422418e-01]])

In [11]:
X_train_scaled, X_test_scaled = (scaler.transform(X_train), scaler.transform(X_test))

In [12]:
from sklearn.linear_model import Lasso

model = Lasso()
model_scaled = Lasso()

model.fit(X_train, y_train)
model_scaled.fit(X_train_scaled, y_train)

C:\Users\WINDOWS TEN PRO\Anaconda3\envs\Platzi - FundamentosAL\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.315977355530281e+17, tolerance: 9458353702123426.0
  positive)


Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [13]:
print(model.score(X_test, y_test))
print(model_scaled.score(X_test_scaled, y_test))

0.5312767243355949
0.5312767259001079


Los modelos de regresión no se ven afectados por el escalamiento de las features. Los de clasificación sí.

# Simplificar las transformaciones con pipelines

In [14]:
from sklearn.pipeline import make_pipeline

model_scaled = make_pipeline(StandardScaler(),Lasso())

In [15]:
model_scaled.fit (X_train, y_train);

In [16]:
print(model_scaled.score(X_test,y_test))

0.5312767259001079


# Crear nuevas features de manera automática

In [17]:
A = np.arange(6).reshape(3,2)

In [18]:
A

array([[0, 1],
       [2, 3],
       [4, 5]])

In [19]:
from sklearn.preprocessing import PolynomialFeatures

In [20]:
transformer = PolynomialFeatures(2)#Argumento: grado de los polinomios
transformer.fit(A)
transformer.transform(A)

array([[ 1.,  0.,  1.,  0.,  0.,  1.],
       [ 1.,  2.,  3.,  4.,  6.,  9.],
       [ 1.,  4.,  5., 16., 20., 25.]])

PolynomialFeatures transforma una matriz $(A1,A2)$ a $(1,A1,A2,A1^2,A1\cdot A2,A2^2)$



In [21]:
transformer.fit_transform(A)

array([[ 1.,  0.,  1.,  0.,  0.,  1.],
       [ 1.,  2.,  3.,  4.,  6.,  9.],
       [ 1.,  4.,  5., 16., 20., 25.]])

In [22]:
X.shape

(4104, 6)

In [23]:
transformer = PolynomialFeatures(2)
transformer.fit_transform(X).shape

(4104, 28)

In [24]:
model_poly = make_pipeline(PolynomialFeatures(2), Lasso())
model_poly.fit(X_train, y_train)
model_poly.score(X_test, y_test)

C:\Users\WINDOWS TEN PRO\Anaconda3\envs\Platzi - FundamentosAL\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.624902010743411e+19, tolerance: 9458353702123426.0
  positive)


0.5885067044406272

## Crear features categoricas

In [25]:
d = pd.DataFrame([['Chile','Colombia','Colombia','Venezuela'],['hombre','mujer','hombre','mujer']])
d = d.T
d.columns = pd.Index(['pais','genero'])
d

,pais,genero
0,Chile,hombre
1,Colombia,mujer
2,Colombia,hombre
3,Venezuela,mujer


In [26]:
pd.get_dummies(d)

,pais_Chile,pais_Colombia,pais_Venezuela,genero_hombre,genero_mujer
0,1,0,0,1,0
1,0,1,0,0,1
2,0,1,0,1,0
3,0,0,1,0,1


## Encoding Binario

In [27]:
!pip install category_encoders

In [37]:
categoricals = pd.read_csv('..\..\..\DataSets\Resultados intermedios peliculas DS\categoricals.csv')

In [38]:
categoricals.head()

,Unnamed: 0,actor_1_name,director_name
0,0,CCH Pounder,James Cameron
1,1,Doug Walker,Doug Walker
2,2,Johnny Depp,Gore Verbinski
3,3,Christoph Waltz,Sam Mendes
4,4,Tom Hardy,Christopher Nolan


In [39]:
categoricals = categoricals.reset_index(drop=True).fillna(0)

In [40]:
categoricals.shape

(4104, 3)

In [41]:
X_binenc = pd.concat([X, categoricals], axis=1)

In [42]:
X_binenc.head()

,production_budget,title_year,aspect_ratio,duration,budget,imdb_score,Unnamed: 0,actor_1_name,director_name
0,425000000.0,2009.000000,1.780000,178.000000,2.370000e+08,7.9,0,CCH Pounder,James Cameron
1,306000000.0,2002.130733,2.126976,108.577186,4.045539e+07,7.1,1,Doug Walker,Doug Walker
2,300000000.0,2007.000000,2.350000,169.000000,3.000000e+08,7.1,2,Johnny Depp,Gore Verbinski
3,300000000.0,2015.000000,2.350000,148.000000,2.450000e+08,6.8,3,Christoph Waltz,Sam Mendes
4,275000000.0,2012.000000,2.350000,164.000000,2.500000e+08,8.5,4,Tom Hardy,Christopher Nolan


In [44]:
import category_encoders as ce
encoder = ce.BinaryEncoder(cols=['actor_1_name','director_name'])

In [46]:
encoder.fit_transform(X_binenc).shape

(4104, 31)

In [47]:
X_binenc = encoder.fit_transform(X_binenc)

In [48]:
Xb_train, Xb_test, y_train, y_test = train_test_split(X_binenc, y)

In [59]:
model_binenc = Lasso()
model = Lasso()

In [60]:
model_binenc.fit(Xb_train,y_train)
model.fit(X_train, y_train)

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [61]:
print(model_binenc.score(Xb_test,y_test))
print(model.score(X_test,y_test))

0.5807863015169626
-0.007220564649830452
